In [2]:
import velocyto as vcy
import numpy as np
import pandas as pd
import inspect
import matplotlib.pyplot as plt

In [3]:
import random
data_delta={}
for smooth_k in [1,2,3]:
    for n in [1,2,3,5,10,20]:
        vlm = vcy.VelocytoLoom('./path/to/LiveCell_3BUJG.loom')
        vlm.filter_cells(bool_array=vlm.initial_Ucell_size > np.percentile(vlm.initial_Ucell_size, 0.1))
        #vlm.filter_cells(bool_array=vlm.initial_Ucell_size < np.percentile(vlm.initial_Ucell_size, 1))
        #vlm.filter_cells(bool_array=vlm.initial_Ucell_size > 1)
        #vlm.set_clusters(vlm.ca["ClusterName"])
        vlm.score_detection_levels(min_expr_counts=10, min_cells_express=10) # 30,20
        vlm.filter_genes(by_detection_levels=True)
        vlm.score_cv_vs_mean(500, plot=False, max_expr_avg=10)#500
        vlm.filter_genes(by_cv_vs_mean=True)

        #vlm.filter_genes(by_custom_array=np.array([g in ccg for g in vlm.ra["Gene"]]))

        vlm.normalize("S", size=True, log=True)
        vlm._normalize_S(relative_size=vlm.S.sum(0),
                     target_size=vlm.S.sum(0).mean())
        vlm._normalize_U(relative_size=vlm.U.sum(0),
                     target_size=vlm.U.sum(0).mean())
        vlm.perform_PCA()
        vlm.knn_imputation(n_pca_dims=3, k=smooth_k, balanced=True)
        vlm.fit_gammas(limit_gamma=False, fit_offset=False)
        vlm.predict_U()
        vlm.calculate_velocity()
        vlm.calculate_shift(assumption="constant_velocity", delta_t=1)#constant_velocity
        vlm.extrapolate_cell_at_t(delta_t=1)


        bh_tsne = TSNE(perplexity=10)
        vlm.ts = bh_tsne.fit_transform(vlm.pcs[:, :2])
        
        coords = [i.split(":")[1][:-1] for i in vlm.ca["CellID"]]
        coord_index = []
        for i in coords:
            seq = i
            lib_id = Sample_ID_dict[seq[:4]].replace(".","_")
            smp_id = seq[4:]
            new_col = lib_id+"_"+smp_id
            coord_index.append(new_col)
        ump = ump.loc[coord_index]
        
        vlm.ts = ump.values # Umap!!!!!
        
        vlm.set_clusters([random.sample(["A","B"],1)[0] for i in range(len(vlm.ca["CellID"]))])
        vlm.estimate_transition_prob(hidim="Sx_sz_t", embed="ts", transform="sqrt", psc=1,n_neighbors=n, knn_random=False)#, sampled_fraction=0.5)
        vlm.calculate_embedding_shift(sigma_corr = 1, expression_scaling=True)
        coord = pd.DataFrame(vlm.embedding,columns=["X","Y"],index=[i.split(":")[1][:-1] for i in vlm.ca["CellID"]])
        coord_index = []
        for i in coord.index:
            seq = i
            lib_id = Sample_ID_dict[seq[:4]].replace(".","_")
            smp_id = seq[4:]
            new_col = lib_id+"_"+smp_id
            coord_index.append(new_col)
        coord.index = coord_index
        cat_coord = pd.concat([coord,sheet],axis=1)
        cat_coord = cat_coord.dropna()
        fig = plt.figure(figsize=(1.5,1.5))
        ax = fig.add_subplot(111)
        import matplotlib as mpl
        font = {'family' : 'Arial','size': 7}
        mpl.rc('font', **font)
        ax.set_axisbelow(True)
        ax.tick_params(labelbottom="off",bottom="on", labelleft="on",left="on", length=2,width=1, color=black)
        for j in ["left","bottom"]:
            ax.spines[j].set_color(black)
            ax.spines[j].set_linewidth(1)
        for j in ["top","right"]:
            ax.spines[j].set_color("w")
        plt.xticks([0,0.25,0.5,0.75,1])
        plt.yticks([0,0.25,0.5,0.75,1])
        plt.xlabel("")
        plt.ylabel("")
        ax.xaxis.grid(True, which='major', linestyle='--', color ="lightgray",linewidth=0.5)
        ax.yaxis.grid(True, which='major', linestyle='--', color ="lightgray",linewidth=0.5)
        sc_list = [i for i in ump.index if i[:5]=="Hi015"]
        for n2,f in enumerate(["2","3","4","5"]):
            sc_list_f = [i for i in sc_list if i.split(".")[1][0]==f]
            ax.scatter(x=ump.loc[sc_list_f,0],y=ump.loc[sc_list_f,1],s=15,color=["#778DA9","#ACB7C3","#CDA0A3","#BA6668"][n2],edgecolor=black,linewidth=0.5)
        lc_list = [i for i in ump.index if i[:5]!="Hi015"]
        for n2,t in enumerate([0,2.5,5,7.5]):
            lc_list_t = [i for i in lc_list if sheet.loc[i,"Time"]==t]
            ax.scatter(x=ump.loc[lc_list_t,0],y=ump.loc[lc_list_t,1],s=12.5,
                       color=["#778DA9","#ACB7C3","#CDA0A3","#BA6668"][n2],edgecolor=black,linewidth=0.5,
                      marker="o")
        data_delta["smoothK_"+str(smooth_k)+"_n_"+str(n)+"_X"] = vlm.delta_embedding[:, 0]
        data_delta["smoothK_"+str(smooth_k)+"_n_"+str(n)+"_Y"] = vlm.delta_embedding[:, 1]
        ax.quiver(vlm.embedding[:, 0], vlm.embedding[:, 1],
                vlm.delta_embedding[:, 0], vlm.delta_embedding[:, 1],
                headaxislength=2.5, headlength=3, headwidth=3.5, minlength=0.15,width=0.005,zorder=3
                )
        plt.xlim(-0.2,1.2)
        plt.ylim(-0.2,1.2)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        #plt.axis("off")
        time=[0,2.5,5,7.5]
        time=[7.5,0,2.5,5][::-1]
        plt.show()